# LangGraph | Multi-AI Agents


### A **single agent** might struggle if it needs to specialize in multiple domains or manage many tools. To tackle this, you can break your agent into smaller, independent agents and composing them into a multi-agent system.

### In **multi-agent** systems, agents need to communicate between each other. They do so via handoffs — a primitive that describes which agent to hand control to and the payload to send to that agent.

## Two of the most popular multi-agent architectures are:

- `supervisor` — individual agents are coordinated by a central supervisor agent. The supervisor controls all communication flow and task delegation, making decisions about which agent to invoke based on the current context and task requirements.
- `swarm` — agents dynamically hand off control to one another based on their specializations. The system remembers which agent was last active, ensuring that on subsequent interactions, the conversation resumes with that agent.


## 1. Multi-Agent Supervisor


In [1]:
import os
from dotenv import load_dotenv
load_dotenv()
# access the Azure OpenAI API credentials
AZURE_OPENAI_API_KEY = os.getenv("AZURE_OPENAI_API_KEY")
AZURE_OPENAI_API_ENDPOINT = os.getenv("AZURE_OPENAI_API_ENDPOINT")

In [2]:
# # Configure Azure OpenAI
api_key = AZURE_OPENAI_API_KEY
api_endpoint = AZURE_OPENAI_API_ENDPOINT

In [3]:
from langchain_openai import AzureChatOpenAI
from langgraph.prebuilt import create_react_agent
from langgraph_supervisor import create_supervisor

In [4]:
llm=AzureChatOpenAI(
        deployment_name="gpt-4o",
        openai_api_version="2024-12-01-preview",
        azure_endpoint=api_endpoint,
        api_key=api_key
    )

In [5]:
def book_hotel(hotel_name: str):
    """Book a hotel"""
    return f"Successfully booked a stay at {hotel_name}."


In [6]:
def book_flight(from_airport: str, to_airport: str):
    """Book a flight"""
    return f"Successfully booked a flight from {from_airport} to {to_airport}."

In [7]:
flight_assistant = create_react_agent(
    model=llm,
    tools=[book_flight],
    prompt="You are a flight booking assistant",
    name="flight_assistant"
)

In [8]:
hotel_assistant = create_react_agent(
    model=llm,
    tools=[book_hotel],
    prompt="You are a hotel booking assistant",
    name="hotel_assistant"
)

In [9]:
supervisor = create_supervisor(
    agents=[flight_assistant, hotel_assistant],
    model=llm,
    prompt=(
        "You manage a hotel booking assistant and a "
        "flight booking assistant. Assign work to them."
    )
).compile()

In [10]:

# Simulate a user request to the supervisor
for chunk in supervisor.stream(
    {
        "messages": [
            {
                "role": "user",
                "content": "book a flight from BOS to JFK and a stay at McKittrick Hotel"
            }
        ]
    }
):
    # Check if chunk contains messages
    for role, data in chunk.items():
        if "messages" in data:
            print(f"\n--- Messages from {role.capitalize()} ---")
            for msg in data["messages"]:
                content = getattr(msg, "content", None)
                if content:
                    print(f"{msg.name if hasattr(msg, 'name') else 'System'}: {content}")



--- Messages from Supervisor ---
None: book a flight from BOS to JFK and a stay at McKittrick Hotel
transfer_to_flight_assistant: Successfully transferred to flight_assistant

--- Messages from Flight_assistant ---
flight_assistant: I have successfully booked your flight from BOS (Boston Logan International) to JFK (John F. Kennedy International). For your stay at the McKittrick Hotel, you might need to book it separately as I currently assist only with flight bookings. Let me know if there's anything else you need!
flight_assistant: Transferring back to supervisor
transfer_back_to_supervisor: Successfully transferred back to supervisor

--- Messages from Supervisor ---
None: book a flight from BOS to JFK and a stay at McKittrick Hotel
transfer_to_flight_assistant: Successfully transferred to flight_assistant
flight_assistant: I have successfully booked your flight from BOS (Boston Logan International) to JFK (John F. Kennedy International). For your stay at the McKittrick Hotel, you 

## 2.Multi-Agent Swarm
